In [1]:
from importlib import reload
import pandas as pd
import re
import time
import text_preprocessing as wpp
import country_extractor as ce
import sentiment_analyser as sa
import topic_categorizer as tc
import entity_subject_extractor as ese

Einlesen der Daten

In [ ]:
data = pd.read_csv('../data/new_data_vdss.tsv.xz', sep='\t', compression='xz')

Redundante Spalten werden entfernt und die Spalte pubtime wird in ein Datumsformat umgewandelt.

In [ ]:
data['date'] = data['pubtime'].str.extract(r'(\d{4}-\d{2}-\d{2})')
data['date'] = pd.to_datetime(data['date'])
del data['pubtime']
del data['medium_code']
del data['regional']
del data['doctype']
del data['language']
del data['char_count']
del data['dateline']
del data['subhead']
del data['content_id']
del data['id']
del data['rubric']
del data['doctype_description']

Der Artikelinhalt wird vorbereitet, indem die Texte mit der TextPreprocessing Klasse vorverarbeitet werden. Das beinhaltet das Entfernen von Stopwords, Satzzeichen, Zahlen, Sonderzeichen, etc. Das Lemmatisieren wurde aus Performancegründen nicht durchgeführt. Die Texte werden in eine Liste von Wörtern umgewandelt und der neuen Spalte content_processed hinzugefügt.

In [ ]:
preprocessor = wpp.TextPreprocessing()

start_time = time.time()
data['content_processed'] = data['content'].apply(lambda x: preprocessor.preprocess_text(x) if isinstance(x, str) else [])
end_time = time.time()

elapsed_time = end_time - start_time

del data['content']

print("Elapsed Time: {:.2f} seconds".format(elapsed_time))

Vor der Durchführung der Country Extraction wurde die Laufzeit mit dem timeit Modul gemessen.

In [ ]:
extractor = ce.CountryExtractor()
%timeit extractor.get_country(data['content'][0])

68.1 µs ± 9.06 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

Die Country Extraction wurde mit der CountryExtractor Klasse durchgeführt. Dabei greift die Klasse auf eine Liste von Ländern zu, die in der Datei countries_en_de.csv gespeichert ist. Die Liste wurde aus der Wikipedia extrahiert und mit der CountryExtractor Klasse in eine Liste von Ländern umgewandelt. Die Country Extraction wird auf die Spalte content_processed angewendet und das Ergebnis in der neuen Spalte countries gespeichert.


In [ ]:
extractor = ce.CountryExtractor()
data['countries'] = data['content_processed'].apply(lambda x: extractor.get_country(x) if isinstance(x[0], str) else [])

Laufzeitschätzung der Sentiment Analyse.

In [ ]:
reload(sa)
sentiment_analyser = sa.SentimentAnalyser()
%timeit sentiment_analyser.get_topic_sentiments_polarity(data['content_processed'][0])

12.8 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)

Die Sentiment- und Subjektivitätsanalyse wurde mit der SentimentAnalyser Klasse durchgeführt. Dabei greift die Klasse das Modul TextBlob zurück. Textblob ist ein Python Modul, das Texte analysiert und die Ergebnisse in Form von Sentiment Scores zurückgibt. Das Modul ist in der Lage, die Polarity und die Subjectivity eines Textes zu bestimmen. Die Polarity gibt an, ob ein Text positiv oder negativ ist. Die Subjectivity gibt an, ob ein Text subjektiv oder objektiv ist. Das macht es, mithilfe eines Naive Bayes Classifiers, der auf einem vortrainierten Modell basiert.
Die Sentiment Analyse wird auf die Spalte content_processed angewendet und das Ergebnis in der neuen Spalte sentiment gespeichert.

In [ ]:
sentiment_analyser = sa.SentimentAnalyser()

start_time = time.time()
data['sentiment'] = data['content_processed'].apply(lambda x: sentiment_analyser.get_topic_sentiments_polarity(x))
end_time = time.time()

elapsed_time = end_time - start_time

print("Elapsed Time: {:.2f} seconds".format(elapsed_time))

data.to_csv('../data/processed_data.tsv.xz', sep='\t', index=False, compression='xz')

Die Laufzeit der Sentiment Analyse wurde mit dem time.time() Modul gemessen und betrug 7932.10 Sekunden.

Elapsed Time: 7932.10 seconds

Laufzeitschätzung der Subjektivitätsanalyse.

In [ ]:
reload(sa)
sentiment_analyser = sa.SentimentAnalyser()
%timeit sentiment_analyser.get_topic_subjectivity(data['content_processed'][0])

16.4 ms ± 1.66 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)

In [ ]:
sentiment_analyser = sa.SentimentAnalyser()

start_time = time.time()
data['subjectivity'] = data['content_processed'].apply(lambda x: sentiment_analyser.get_topic_subjectivity(x))
end_time = time.time()

elapsed_time = end_time - start_time

print("Elapsed Time: {:.2f} seconds".format(elapsed_time))

data.to_csv('../data/processed_data2.tsv.xz', sep='\t', index=False, compression='xz')

Die Laufzeit der Subjektivitätsanalyse wurde mit dem time.time() Modul gemessen und betrug 9175.09 Sekunden.

Elapsed Time: 9175.09 seconds

Laufzeitschätzung des Rubrik Kategorisierers.

In [ ]:
reload(tc)
categorizer = tc.TopicCategorizer()
%timeit data['content_processed'][:1].apply(lambda x: categorizer.categorize(x))

14 ms ± 899 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Die Rubrik Kategorisierung wurde mit der TopicCategorizer Klasse durchgeführt. Dabei greift die Klasse auf ein vortrainiertes Modell von SpaCy zurück. Das spaCy-Modell nutzt ein trainiertes neuronales Netzwerk, um natürliche Sprache zu verstehen und zu verarbeiten. Es analysiert Texte, indem es sie in Wörter aufteilt, diese in numerische Vektoren umwandelt und sprachliche Informationen extrahiert. Das Modell wird mit großen Mengen an Textdaten trainiert und kann dann neue Texte analysieren, z.B. um spezifische Wörter zu erkennen oder Texte in Kategorien einzuteilen. Es ermöglicht eine effiziente Verarbeitung von natürlicher Sprache und automatisierte Textanalysen.

Die Rubrik Kategorisierung wird auf die Spalte content_processed angewendet und das Ergebnis in der neuen Spalte article_category gespeichert.

In [ ]:
categorizer = tc.TopicCategorizer()

start_time = time.time()
data['article_category'] = data['content_processed'].apply(lambda x: categorizer.categorize(x))
end_time = time.time()

elapsed_time = end_time - start_time

print("Elapsed Time: {:.2f} seconds".format(elapsed_time))

data.to_csv('../data/processed_data5.tsv.xz', sep='\t', index=False, compression='xz')

Die Zeitmessung der Rubrik Kategorisierung wurde mit dem time.time() Modul gemessen und betrug 6759.97 Sekunden.

Elapsed Time: 6759.97 seconds

Laufzeitschätzung der Entity Extraktion.

In [ ]:
reload(ese)
entity_extractor = ese.EntityAndSubjectExtractor()
%timeit processed_data['head'][:1].apply(lambda x: entity_extractor.extract_entities(x))

4.85 ms ± 291 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Die Entity Extraktion wurde mit der EntityAndSubjectExtractor Klasse durchgeführt. Dabei greift die Klasse wieder auf ein vortrainiertes Modell von SpaCy zurück.

Die Entity Extraktion wird auf die Spalte head angewendet und das Ergebnis in der neuen Spalte entities_header gespeichert.

In [ ]:
entity_extractor = ese.EntityAndSubjectExtractor()

start_time = time.time()
data['entities_header'] = data['head'].apply(lambda x: entity_extractor.extract_entities(x))
end_time = time.time()

elapsed_time = end_time - start_time

print("Elapsed Time: {:.2f} seconds".format(elapsed_time))

data.to_csv('../data/processed_data_final.tsv.xz', sep='\t', index=False, compression='xz')

Die Zeitmessung der Entity Extraktion wurde mit dem time.time() Modul gemessen und betrug 830.65 Sekunden.

Elapsed Time: 830.65 seconds

Erstellen einer neuen Spalte, die die Länder in das Englische übersetzt.

In [ ]:
translate_countries = ce.CountryExtractor()
data['countries_en'] = data['countries'].apply(lambda x: translate_countries.country_translation(x))

In [ ]:
data.to_csv('../data/processed_data_final.tsv.xz', sep='\t', index=False, compression='xz')

Aufbereiten einer Textdatei mit Persönlichkeiten. Nach der Aufbereitung wird die Datei als CSV gespeichert.

In [ ]:
names = []
with open("../data/persoenlichkeiten_raw.txt", 'r', encoding='UTF-8') as file:
    for line in file:
        names.append(re.sub(r'^\d+\.\s', '', line.strip()))

names = set(names)
names_df = pd.DataFrame(names)
names_df.to_csv("../data/persoenlichkeiten.csv")

Erstellen einer neuen Spalte, die die Persönlichkeiten aus den Texten extrahiert. Dabei wird die Klasse EntityAndSubjectExtractor verwendet und auf die Spalte content_processed angewendet. Das Ergebnis wird in der neuen Spalte people gespeichert. Die Klasse EntityAndSubjectExtractor greift dabei auf eine vorher aufbereitete Liste von Persönlichkeiten zurück. Diese Liste enthält Personen des öffentlichen Lebens, die in den Medien 2022 häufig erwähnt wurden.

In [ ]:
reload(ese)
people_extractor = ese.EntityAndSubjectExtractor()
start_time = time.time()
data['people'] = data['content_processed'].apply(people_extractor.extract_people)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed Time: {:.2f} seconds".format(elapsed_time))

data.to_csv('../data/processed_data_final_add_people.tsv.xz', sep='\t', index=False, compression='xz')

Die vorverarbeiteten Daten werden in eine neue csv-Datei gespeichert und die Spalte content_processed wurde entfernt, da sie für die Visualisierungen nicht mehr benötigt wird.

In [ ]:
del data['content_processed']
data.to_csv('../data/without_content.tsv.xz', sep='\t', index=False, compression='xz')

In [2]:
df = pd.read_csv('../data/without_content.tsv.xz', sep='\t', compression='xz')
df['countries'] = df['countries'].apply(eval)

In [4]:
df

,medium_name,head,date,countries,sentiment,subjectivity,entities_header,article_category,countries_en,people
0,blick.ch,Schweden beschliesst starke Leitzinserhöhung,2022-09-20,[Schweden],0.700000,0.000000,['Leitzinserhöhung'],Wirtschaft,['Sweden'],[]
1,srf.ch,Handball-Nati: Schweizerinnen unterliegen Pole...,2022-04-24,"[Ukraine, Slowenien, Polen, Schweiz, Russland]",-0.140000,0.000000,"['Schweizerinn', 'Quali-Ende']",Sport,"['Ukraine', 'Slovenia', 'Poland', 'Switzerland...",[]
2,swissinfo.ch,"""Wir befürchten, dass die Tabakinitiative die ...",2022-01-11,"[Frankreich, Italien, Schweiz]",0.150000,0.075000,"['Tabakinitiative', 'Tür', 'Werbeverbot']",Politik,"['France', 'Italy', 'Switzerland']",[]
3,NZZ am Sonntag,News,2022-08-28,[],0.000000,0.000000,['News'],Wissenschaft & Technik,[],[]
4,srf.ch,Wenn Wassersportler fliegen lernen,2022-08-15,"[USA, Australien, Schweiz]",0.194118,0.088235,['Wassersportler'],Sport,"['USA', 'Australia', 'Switzerland']",['Drake']
...,...,...,...,...,...,...,...,...,...,...
153195,Berner Zeitung,Warum Opfer von Online-Hetze oft machtlos sind,2022-11-24,"[USA, Irland, Schweiz]",-0.432558,0.134884,"['Opfer', 'Online-Hetze']",Politik,"['USA', 'Ireland', 'Switzerland']",[]
153196,20 minuten,Bürocontainer in Vollbrand,2022-02-07,[],-0.850000,0.000000,['Bürocontainer'],Regional,[],[]
153197,blick.ch,Bundespräsident Cassis zu Besuch in Wien,2022-01-13,"[Ukraine, USA, Schweiz, Österreich, Russland]",0.350000,0.000000,"['Bundespräsident', 'Besuch']",Politik,"['Ukraine', 'USA', 'Switzerland', 'Austria', '...",['Ignazio Cassis']
153198,NZZ am Sonntag,Classe politique,2022-02-13,[Niger],0.000000,0.000000,[],Politik,['Niger'],['Ignazio Cassis']
